# Stored Procedure definitions:

## HR Role

- Proc_Add_Employee: Adds an employee to the database

In [1]:
drop proc [Theme_Park].[Proc_Add_Employee]
go
create proc [Theme_Park].[Proc_Add_Employee]
    @fname varchar(20),
    @lname varchar(20),
    @ssn numeric(9,0),
    @gender char,
    @address varchar(50),
    @phone numeric(10,0),
    @date_joined date,
    @dept_id int,
    @role varchar(20),
    @supervisor_ssn numeric(9,0),
    @salaried bit,
    @payrate money,
    @vacation_days decimal(4,2)
AS
    INSERT INTO [Theme_Park].[Employee]
    VALUES
    ( @fname, @lname, @ssn, @gender, @address, @phone, @date_joined, @dept_id, @role, @supervisor_ssn, @salaried, @payrate, @vacation_days );
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.184

In [8]:
exec [Theme_Park].[Proc_Add_Employee] 'Kimmy', 'Schmidt', 177889999, 'F', 'Middletown, OH', 01117777, '2022-10-24', 0, 'Heroine', 177889999, 0, 17.15, 0
go

(1 row affected)

Total execution time: 00:00:00.063

## Customer entity

PROC_CUSTOMER_BUY_TICKET: adds a new ticket purchase to the database

In [1]:
drop proc [Theme_Park].[Proc_Customer_Buy_Ticket]
go
create proc [Theme_Park].[Proc_Customer_Buy_Ticket]
    @date datetime,
    @class varchar(10),
    @price decimal(5,2)
AS
    INSERT INTO [Theme_Park].[Ticket] (Date, Ticket_Class, Price, Reservation)
    VALUES
    (@date, @class, @price, NULL)
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.129

In [2]:
exec [Theme_Park].[Proc_Customer_Buy_Ticket] '2022-10-22', 'Poor', 11.50
go

(1 row affected)

Total execution time: 00:00:00.534

PROC_CUSTOMER_BUY_RESERVATION: buy a new ticket reservation

In [3]:
drop proc [Theme_Park].[Proc_Customer_Buy_Reservations]
GO
create proc [Theme_Park].[Proc_Customer_Buy_Reservations]
    @customer_id int,
    @FirstName varchar(25),
    @LastName varchar(25),
    @Date_of_Visit datetime,
    @Credit_Card_Number bigint,
    @Ticket_Class varchar(10)
AS
    declare @Price decimal(5,2) =
    (select case @Ticket_Class
    when 'Poor' then 11.50
    when 'Premium' then 16.00
    else 13.00
    end)
    insert into [Theme_Park].[Ticket_Reservation]
    (Customer_ID, FirstName, LastName, Date_Placed, Date_of_Visit, Credit_Card_Number, Ticket_Class, Expired, Price, Ticket_ID)
    values
    (@customer_id, @FirstName, @LastName, GETDATE(), @Date_of_Visit, @Credit_Card_Number, @Ticket_Class, 0, @Price, null)
go

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.149

In [4]:
select * from [Theme_Park].[Ticket_Reservation]

(3 rows affected)

Total execution time: 00:00:00.346

Reservation_ID,Customer_ID,FirstName,LastName,Date_Placed,Date_of_Visit,Credit_Card_Number,Ticket_Class,Expired,Price,Ticket_ID
1,1,Karen,Strong,2022-11-10 19:14:26.970,2022-12-25 00:00:00.000,111,Premium,0,11.50,NULL
2,1,Michael,Moorman,2022-11-10 21:01:31.230,2022-11-11 00:00:00.000,4266841488253319,Poor,0,11.50,NULL
3,1,Michael,Moorman,2022-11-10 21:14:15.067,2022-11-11 00:00:00.000,4266841488253319,Premium,0,16.00,NULL


PROC_CUSTOMER_GET_CURRENT_RESERVATIONS: lists all current reservations

In [5]:
drop proc [Theme_Park].[Proc_Customer_Get_Current_Reservations]
GO
create proc [Theme_Park].[Proc_Customer_Get_Current_Reservations]
    @customer_id int
AS
    select Reservation_ID, Date_Placed, Date_of_Visit, Ticket_Class, Price, Ticket_ID, FirstName, LastName
    from [Theme_Park].[Ticket_Reservation]
    where Customer_ID=@customer_id
    and Date_of_Visit >= GETDATE()
    order by Date_Placed asc
go

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.244

In [28]:
drop proc [Theme_Park].[Proc_Customer_Cancel_Reservation]
GO
create proc [Theme_Park].[Proc_Customer_Cancel_Reservation]
    @Reservation_ID int
AS
    delete from [Theme_Park].[Ticket_Reservation]
    where Reservation_ID=@Reservation_ID
go

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the procedure 'Theme_Park.Proc_Customer_Cancel_Reservation', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.144

In [23]:
insert into [Theme_Park].[Customer]
(name, cc_num, email, visit_count, username)
values
('Joe Blow', 7777666622221111, 'joeblow@uh.edu', 0, 'joe')
go
exec [Theme_Park].[Proc_Customer_Buy_Reservations] 15, 'Joe', 'Blow', '2022-11-03', 10, "Poor"
go

(1 row affected)

: Msg 547, Level 16, State 0, Procedure Theme_Park.Proc_Customer_Buy_Reservations, Line 9
The INSERT statement conflicted with the FOREIGN KEY constraint "FK_Customer_Of_Reservation". The conflict occurred in database "themepark_dev", table "Theme_Park.Customer", column 'customer_id'.

The statement has been terminated.

Total execution time: 00:00:00.193

In [2]:
drop proc [Theme_Park].[Proc_Get_Maintenance_Notifications]
go
create proc [Theme_Park].[Proc_Get_Maintenance_Notifications]
AS
    select * from [Theme_Park].[Maintenance_Notification]
    order by notification_id asc
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.701

In [20]:
drop proc [Theme_Park].[Proc_Mark_Maintenance_Notifications_Read]
go
create proc [Theme_Park].[Proc_Mark_Maintenance_Notifications_Read]
AS
    delete from [Theme_Park].[Maintenance_Notification]
GO

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the procedure 'Theme_Park.Proc_Mark_Maintenance_Notifications_Read', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.231

In [2]:
drop table [Theme_Park].[Maintenance_Notification]
go
CREATE TABLE [Theme_Park].[Maintenance_Notification](
	[notification_id] [int] IDENTITY(1,1) NOT NULL,
	[maintenance_id] [int] NOT NULL,
	[completion] [datetime] NOT NULL,
	[billed_hours] [int] NULL,
	[amount] [smallmoney] NULL,
    [maint_description] varchar(50) NULL
) ON [PRIMARY]
GO

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the table 'Theme_Park.Maintenance_Notification', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.132

In [27]:
drop proc [Theme_Park].[Proc_Aggregate_Ticket_Sales_By_Weekday]
GO
create proc [Theme_Park].[Proc_Aggregate_Ticket_Sales_By_Weekday]
    @year int
as
    SELECT Ticket_Class, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 0 THEN 1 END) AS SUNDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 1 THEN 1 END) AS MONDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 2 THEN 1 END) AS TUESDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 3 THEN 1 END) AS WEDNESDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 4 THEN 1 END) AS THURSDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 5 THEN 1 END) AS FRIDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 6 THEN 1 END) AS SATURDAY
    from 
    [Theme_Park].[Ticket]
    where
    DATEPART(YEAR,Date) = @year
    group by Ticket_Class
    order by Ticket_Class asc
go

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the procedure 'Theme_Park.Proc_Aggregate_Ticket_Sales_By_Weekday', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:06.793

In [18]:
drop proc [Theme_Park].[Proc_Aggregate_Ticket_Sales_By_Week_For_Given_Year]
GO
create proc [Theme_Park].[Proc_Aggregate_Ticket_Sales_By_Week_For_Given_Year]
    @year int
as
    SELECT
        DATEPART(WEEK, Date) AS WEEK_NUMBER,
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 0 THEN 1 END) AS SUNDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 1 THEN 1 END) AS MONDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 2 THEN 1 END) AS TUESDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 3 THEN 1 END) AS WEDNESDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 4 THEN 1 END) AS THURSDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 5 THEN 1 END) AS FRIDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 6 THEN 1 END) AS SATURDAY
    from 
    [Theme_Park].[Ticket]
    where
    DATEPART(YEAR,Date) = @year
    group by
    DATEPART(Week, Date)
    order by
    DATEPART(Week, Date) asc
go

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the procedure 'Theme_Park.Proc_Aggregate_Ticket_Sales_By_Week_For_Given_Year', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.121

In [19]:
exec [Theme_Park].[Proc_Aggregate_Ticket_Sales_By_Week_For_Given_Year] 2022

Warning: Null value is eliminated by an aggregate or other SET operation.

(22 rows affected)

Total execution time: 00:00:00.083

WEEK_NUMBER,SUNDAY,MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY
20,0,0,8,6,2,6,4
21,0,6,4,8,4,5,8
22,0,4,9,2,3,9,2
23,0,9,4,4,3,1,8
24,0,7,10,6,2,5,7
25,0,11,6,6,8,6,4
26,0,6,6,7,13,6,3
27,0,12,10,5,5,5,9
28,0,8,13,3,4,12,7
29,0,15,5,9,3,5,7


In [20]:
drop proc [Theme_Park].[Proc_Aggregate_Ticket_Sales_For_Given_Week_For_Given_Year]
GO
create proc [Theme_Park].[Proc_Aggregate_Ticket_Sales_For_Given_Week_For_Given_Year]
    @year int,
    @week int
as
    SELECT  
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 0 THEN 1 END) AS SUNDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 1 THEN 1 END) AS MONDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 2 THEN 1 END) AS TUESDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 3 THEN 1 END) AS WEDNESDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 4 THEN 1 END) AS THURSDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 5 THEN 1 END) AS FRIDAY, 
        COUNT(CASE WHEN DATEPART(WEEKDAY,Date) = 6 THEN 1 END) AS SATURDAY
    from 
    [Theme_Park].[Ticket]
    where
    DATEPART(YEAR,Date) = @year
    and
    DATEPART(WEEK,Date) = @week
go

Commands completed successfully.

: Msg 137, Level 15, State 2, Line 4
Must declare the scalar variable "@year".

: Msg 137, Level 15, State 2, Line 18
Must declare the scalar variable "@year".

Total execution time: 00:00:00.291

In [11]:
exec [Theme_Park].[Proc_Aggregate_Ticket_Sales_This_Week_For_Given_Year] 2022

(1 row affected)

Total execution time: 00:00:00.099

SUNDAY,MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY
0,0,0,0,0,0,0


In [17]:
exec [Theme_Park].[Proc_Aggregate_Ticket_Sales_For_Given_Week_For_Given_Year] 2021, 22

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.064

SUNDAY,MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY
0,5,8,8,12,7,10


In [28]:
exec [Theme_Park].[Proc_Aggregate_Ticket_Sales_By_Weekday] 2022

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.232

Ticket_Class,SUNDAY,MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY
Normal,0,10,9,5,5,2,11
Poor,0,3,6,3,23,4,3
Premium,0,4,3,4,6,2,4


In [31]:
select * from [Theme_Park].[Ticket] where DATEPART(YEAR,Date) = 2021 and DATEPART(WEEKDAY,Date) = 1
order by Date asc

(21 rows affected)

Total execution time: 00:00:00.078

Ticket_ID,Date,Ticket_Class,Price,Reservation
180,2021-01-10 01:02:52.000,Normal,11.50,NULL
136,2021-01-17 16:05:51.000,Premium,11.50,NULL
213,2021-01-17 22:51:55.000,Normal,11.50,NULL
222,2021-01-24 11:14:42.000,Poor,11.50,NULL
161,2021-01-24 21:13:24.000,Poor,11.50,NULL
165,2021-03-07 08:59:19.000,Poor,11.50,NULL
185,2021-03-14 07:40:42.000,Normal,11.50,NULL
143,2021-03-21 16:36:33.000,Normal,11.50,NULL
200,2021-04-25 01:58:19.000,Normal,11.50,NULL
131,2021-05-23 00:53:23.000,Poor,11.50,NULL
